# Facebook: Predicting Check Ins

### Group: Tinh Nguyen, Joelynn Deng, AC Dela Paz, Suman Tripathy

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, RidgeClassifier
np.random.seed(30)

In [12]:
#Read in the data
fb_train = pd.read_csv("../../datasets/Facebook/train.csv")
fb_test = pd.read_csv("../../datasets/Facebook/test.csv")
fb_train.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [35]:
X = fb_train.drop(['place_id'], axis=1)
y = fb_train['place_id']
fb_train.groupby('place_id').mean()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [36]:
randomx = np.random.random()*10
print(randomx)
randomy = np.random.random()*10
print(randomy)

while ((randomx + .25 > 10) or (randomy + .25 > 10)):
    randomx = np.random.random()*10
    randomy = np.random.random()*10

7.580753568836803
7.249712698123841


In [54]:
X_train.head()

,row_id,x,y,accuracy,time
305587,305587,5.1050,6.4115,172,362163
93140,93140,2.6905,5.8470,5,369458
781106,781106,4.4467,9.9711,67,560596
1007270,1007270,6.8005,4.7053,57,110013
65266,65266,9.6079,4.5557,67,743654


In [ ]:
X_vals = (X_train['x'] > randomx) & (X_train['x'] < (randomx + .25))
Y_vals = (X_train['y'] > randomy) & (X_train['y'] < (randomy + .25))
true_vals = X_vals & Y_vals
true_vals.shape
new_data = X_train.where(true_vals)
new_data.head()

In [ ]:
def cv_stats(cv_score):
    """ 
    Takes in the output of cross_val_score
    Returns the mean and standard deviation in a readable format
    """
    mean = np.mean(cv_score)
    std = np.std(cv_score)
    return mean, std

# K Nearest Neighbors Model

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

knn = KNN(3)
clf_score = cross_val_score(knn, X_train, y_train, cv = 5)
clf_score 

/Users/suman/anaconda3/lib/python3.4/site-packages/sklearn/cross_validation.py:417: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


array([  4.70874084e-05,   1.00290843e-04,   6.00827941e-05,
         1.03607617e-04,   7.15273473e-05])

# Linear Regression Model


In [ ]:
# Initialize linear regression
clf = linear_model.LinearRegression()
clf.fit(X_train, y_train)


In [ ]:
#Score Linear Regression

train_r2 = clf.score(X_train, y_train) # fill out linreg
test_r2 = clf.score(X_test, y_test)# fill out linreg

print("Train accuracy : ", train_r2)
print("Test accuracy : ", test_r2)

In [ ]:
y_pred = clf.predict(X_test) # predict on test set

fig, ax = plt.subplots()
ax.scatter(y_test, y_pred)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=4)
ax.set_xlabel('Test')
ax.set_ylabel('Predicted')
plt.title('Comparison of Actual vs Predicted Values')
plt.show()

#Logistic Regression Model


In [ ]:
# iterate through a range of penalty values and evaluate how well LogisticRegression does on average
penalty = 0.01
while penalty < 10:
    clf = LogisticRegression(C = penalty)
    score = cross_val_score(clf, X_train, y_train, cv = 5)
    stats = cv_stats(score)
    print("For penalty = ", penalty, "Average, std = ", stats)
    penalty += 0.001
    
# initialize and fit logistic regression
clf = linear_model.LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
train_acc = # score Logistic Regression on train set
test_acc = # score Logistic Regression on test set


print("Train accuracy : ", train_acc)
print("Test accuracy : ", test_acc)

# Ridge Regression Model


In [58]:
# iterate through a range of penalty values and evaluate how well RidgeClassifier does on average
penalty = 0.01
while penalty < 10:
    clf = RidgeClassifier(alpha = penalty)
    score = cross_val_score(clf, X_train, y_train, cv = 5)
    stats = cv_stats(score)
    print("For penalty = ", penalty, "Average, std = ", stats)
    penalty += 0.001

NameError: name 'RidgeClassifier' is not defined

# Best Model
## K Nearest Neighbors

In [ ]:
# initialize KNN here
clf = KNN(12)
clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train) # score KNN on train set
test_acc = clf.score(X_test, y_test) # score KNN on test set


print("Train accuracy : ", train_acc)
print("Test accuracy : ", test_acc)